# Introduction to Cross Validation

In this lecture series we will do a much deeper dive into various methods of cross-validation. As well as a discussion on the general philosphy behind cross validation. A nice official documentation guide can be found here: https://scikit-learn.org/stable/modules/cross_validation.html

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../DATA/Advertising.csv')

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


----
----
----
## Train | Test Split Procedure 

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Test Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Test Data (by creating predictions and comparing to Y_test)
7. Adjust Parameters as Necessary and repeat steps 5 and 6

In [4]:
# Create X and y
X = df.drop('sales', axis=1)
y = df['sales']

In [5]:
# Train Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=101)

In [6]:
# Scale data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

**Create Model**

In [7]:
from sklearn.linear_model import Ridge

In [8]:
model = Ridge(alpha=100)

In [9]:
model.fit(scaled_X_train, y_train)

Ridge(alpha=100)

In [10]:
y_pred = model.predict(scaled_X_test)

**evaluation**

In [11]:
from sklearn.metrics import mean_squared_error

In [12]:
mean_squared_error(y_test,y_pred)

7.34177578903413

**Adjust Parameters and Re-evaluate**

In [13]:
model_two = Ridge(alpha=1)

In [14]:
model_two.fit(scaled_X_train, y_train)

Ridge(alpha=1)

In [15]:
y_pred_two = model_two.predict(scaled_X_test)

**another evaluation**

In [16]:
mean_squared_error(y_test, y_pred_two)

2.3190215794287523

Much better! We could repeat this until satisfied with performance metrics. (We previously showed RidgeCV can do this for us, but the purpose of this lecture is to generalize the CV process for any model).

----
----
----
## Train | Validation | Test Split Procedure 

This is often also called a "hold-out" set, since you should not adjust parameters based on the final test set, but instead use it *only* for reporting final expected performance.

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Validation/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Eval Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Evaluation Data (by creating predictions and comparing to Y_eval)
7. Adjust Parameters as Necessary and repeat steps 5 and 6
8. Get final metrics on Test set (not allowed to go back and adjust after this!)

In [17]:
# Create X and y
X = df.drop('sales', axis=1)
y = df['sales']

In [18]:
######################################################################
#### SPLIT TWICE! Here we create TRAIN | VALIDATION | TEST  #########
####################################################################

from sklearn.model_selection import train_test_split

# 70% of data is training data, set aside other 30%
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=.3, random_state=101)

# Remaining 30% is split into evaluation and test sets
# Each is 15% of the original data size
X_eval, X_test, y_eval, y_test = train_test_split(X_other, y_other, test_size=.5, random_state=101)

In [19]:
# SCALE DATA

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_eval = scaler.transform(X_eval)
scaled_X_test = scaler.transform(X_test)

**create the model**

In [20]:
from sklearn.linear_model import Ridge

In [21]:
model_one = Ridge(alpha=100)

In [22]:
model_one.fit(scaled_X_train, y_train)

Ridge(alpha=100)

In [23]:
y_eval_pred = model.predict(scaled_X_eval)

**evaluation**

In [24]:
from sklearn.metrics import mean_squared_error

In [25]:
mean_squared_error(y_eval, y_eval_pred)

7.320101458823871

**Adjust Parameters and Re-evaluate**

In [26]:
model_two = Ridge(alpha=1)

In [27]:
model_two.fit(scaled_X_train,y_train)

Ridge(alpha=1)

In [28]:
new_eval_pred = model_two.predict(scaled_X_eval)

**Another Evaluation**

In [29]:
mean_squared_error(y_eval, new_eval_pred)

2.383783075056986

**Final Evaluation (Can no longer edit parameters after this!)**

In [30]:
y_final_test_pred = model_two.predict(scaled_X_test)

In [31]:
mean_squared_error(y_test, y_final_test_pred)

2.2542600838005176

----
----
----
## Cross Validation with cross_val_score

----

<img src="grid_search_cross_validation.png">

----

In [32]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

In [33]:
# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [34]:
# SCALE DATA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

**Create the model**

In [35]:
model = Ridge(alpha=100)

In [36]:
from sklearn.model_selection import cross_val_score

In [37]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model, scaled_X_train, y_train, scoring='neg_mean_squared_error',cv=5)

In [38]:
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [39]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

8.215396464543606

**Adjust model based on metrics**

In [40]:
model = Ridge(alpha=1)

In [41]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model, scaled_X_train, y_train, scoring='neg_mean_squared_error', cv=5)

In [42]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

3.344839296530695

**Final Evaluation (Can no longer edit parameters after this!)**

In [43]:
# Need to fit the model first!
model.fit(scaled_X_train,y_train)

Ridge(alpha=1)

In [44]:
y_final_test_pred = model.predict(scaled_X_test)

In [45]:
mean_squared_error(y_test,y_final_test_pred)

2.3190215794287523

----
----
----

# Cross Validation with cross_validate

The cross_validate function differs from cross_val_score in two ways:

It allows specifying multiple metrics for evaluation.

It returns a dict containing fit-times, score-times (and optionally training scores as well as fitted estimators) in addition to the test score.

For single metric evaluation, where the scoring parameter is a string, callable or None, the keys will be:
        
        - ['test_score', 'fit_time', 'score_time']

And for multiple metric evaluation, the return value is a dict with the following keys:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score is set to False by default to save computation time. To evaluate the scores on the training set as well you need to be set to True.

In [46]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

**create the model**

In [47]:
model = Ridge(alpha=100)

In [48]:
from sklearn.model_selection import cross_validate

In [49]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,scaled_X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [50]:
scores

{'fit_time': array([0.00117612, 0.00068998, 0.00052595, 0.00043201, 0.00042415]),
 'score_time': array([0.0008781 , 0.00064731, 0.00060701, 0.00056291, 0.00055671]),
 'test_neg_mean_absolute_error': array([-2.31243044, -1.74653361, -2.56211701, -2.01873159, -2.27951906]),
 'test_neg_mean_squared_error': array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
         -8.38562723]),
 'test_max_error': array([ -6.44988486,  -5.58926073, -10.33914027,  -6.61950405,
         -7.75578515])}

In [51]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.001176,0.000878,-2.312430,-9.325530,-6.449885
1,0.000690,0.000647,-1.746534,-4.944962,-5.589261
2,0.000526,0.000607,-2.562117,-11.396652,-10.339140
3,0.000432,0.000563,-2.018732,-7.024211,-6.619504
4,0.000424,0.000557,-2.279519,-8.385627,-7.755785


In [52]:
pd.DataFrame(scores).mean()

fit_time                        0.000650
score_time                      0.000650
test_neg_mean_absolute_error   -2.183866
test_neg_mean_squared_error    -8.215396
test_max_error                 -7.350715
dtype: float64

**Adjust model based on metrics**

In [53]:
model = Ridge(alpha=1)

In [54]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,scaled_X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [55]:
pd.DataFrame(scores).mean()

fit_time                        0.000697
score_time                      0.000686
test_neg_mean_absolute_error   -1.319685
test_neg_mean_squared_error    -3.344839
test_max_error                 -5.161145
dtype: float64

**Final Evaluation (Can no longer edit parameters after this!)**

In [56]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=1)

In [57]:
y_final_test_pred = model.predict(X_test)

In [58]:
mean_squared_error(y_test,y_final_test_pred)

2.29872742893374

# Grid Search 

We can search through a variety of combinations of hyperparameters with a grid search. While many linear models are quite simple and even come with their own specialized versions that do a search for you, this method of a grid search will can be applied to *any* model from sklearn, and we will need to use it later on for more complex models, such as Support Vector Machines.

**Data**

In [59]:
df = pd.read_csv("../DATA/Advertising.csv")

In [60]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


**Formating Data**

In [62]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

**Create the model**

In [63]:
from sklearn.linear_model import ElasticNet

In [64]:
base_elastic_net_model = ElasticNet()

## Grid Search

A search consists of:

* an estimator (regressor or classifier such as sklearn.svm.SVC());
* a parameter space;
* a method for searching or sampling candidates;
* a cross-validation scheme 
* a score function.

In [65]:
param_grid = {'alpha':[.1,1,5,10,50,100],
              'l1_ratio':[.1,.5,.7,.95,.99,1]}

In [66]:
from sklearn.model_selection import GridSearchCV

In [67]:
grid_model = GridSearchCV(estimator=base_elastic_net_model,
                         param_grid=param_grid,
                         scoring="neg_mean_squared_error",
                         cv=5,
                         verbose=2 )# show information: 1 give less info and 0 None

In [68]:
grid_model.fit(scaled_X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.7; total time=   0.0s
[CV] END ............................alpha=0.1,

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.95, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=2)

In [69]:
grid_model.best_estimator_

ElasticNet(alpha=0.1, l1_ratio=1)

In [70]:
grid_model.best_params_

{'alpha': 0.1, 'l1_ratio': 1}

In [71]:
grid_model.cv_results_

{'mean_fit_time': array([0.0028306 , 0.00094299, 0.00085001, 0.00083542, 0.0006597 ,
        0.00071321, 0.00065389, 0.00059614, 0.00053697, 0.00050607,
        0.00051146, 0.00045238, 0.0004375 , 0.00043473, 0.00043411,
        0.00044694, 0.00044913, 0.00043035, 0.00043092, 0.00043144,
        0.00042739, 0.00043321, 0.00043111, 0.00047579, 0.00043736,
        0.00043073, 0.00042896, 0.00043006, 0.00042872, 0.00042639,
        0.00042963, 0.00042615, 0.00042801, 0.00042896, 0.00043054,
        0.0004272 ]),
 'std_fit_time': array([2.67750972e-03, 1.30251053e-04, 7.36430802e-05, 1.05965941e-04,
        9.27322619e-06, 2.43591458e-05, 3.79334674e-05, 7.67337520e-06,
        1.94742304e-05, 5.26813352e-06, 2.15672767e-05, 1.92665816e-05,
        1.90734863e-06, 2.87924369e-06, 5.74860766e-06, 1.39091037e-05,
        6.69170917e-06, 4.35203713e-06, 5.35248383e-07, 4.98881975e-06,
        9.72560790e-07, 5.29482523e-06, 5.78409566e-06, 4.24185717e-05,
        3.63336147e-06, 2.21921468e-0

In [72]:
pd.DataFrame(grid_model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002831,2.677510e-03,0.000884,3.881347e-04,0.1,0.1,"{'alpha': 0.1, 'l1_ratio': 0.1}",-3.453021,-1.405190,-5.789125,-2.187302,-4.645576,-3.496043,1.591601,6
1,0.000943,1.302511e-04,0.000503,3.904467e-05,0.1,0.5,"{'alpha': 0.1, 'l1_ratio': 0.5}",-3.325440,-1.427522,-5.595610,-2.163089,-4.451679,-3.392668,1.506827,5
2,0.000850,7.364308e-05,0.000492,6.635997e-05,0.1,0.7,"{'alpha': 0.1, 'l1_ratio': 0.7}",-3.269880,-1.442432,-5.502437,-2.163950,-4.356738,-3.347088,1.462765,4
3,0.000835,1.059659e-04,0.000474,8.337994e-05,0.1,0.95,"{'alpha': 0.1, 'l1_ratio': 0.95}",-3.213052,-1.472417,-5.396258,-2.177452,-4.241080,-3.300052,1.406248,3
4,0.000660,9.273226e-06,0.000365,3.102741e-06,0.1,0.99,"{'alpha': 0.1, 'l1_ratio': 0.99}",-3.208124,-1.478489,-5.380242,-2.181097,-4.222968,-3.294184,1.396953,2
5,0.000713,2.435915e-05,0.000396,1.781266e-05,0.1,1,"{'alpha': 0.1, 'l1_ratio': 1}",-3.206943,-1.480065,-5.376257,-2.182076,-4.218460,-3.292760,1.394613,1
6,0.000654,3.793347e-05,0.000349,1.661324e-05,1,0.1,"{'alpha': 1, 'l1_ratio': 0.1}",-9.827475,-5.261525,-11.875347,-7.449195,-8.542329,-8.591174,2.222939,12
7,0.000596,7.673375e-06,0.000327,2.388949e-06,1,0.5,"{'alpha': 1, 'l1_ratio': 0.5}",-8.707071,-4.214228,-10.879261,-6.204545,-7.173031,-7.435627,2.255532,11
8,0.000537,1.947423e-05,0.000294,7.741228e-06,1,0.7,"{'alpha': 1, 'l1_ratio': 0.7}",-7.920870,-3.549562,-10.024877,-5.379553,-6.324836,-6.639940,2.206213,10
9,0.000506,5.268134e-06,0.000282,3.092465e-06,1,0.95,"{'alpha': 1, 'l1_ratio': 0.95}",-6.729435,-2.591285,-8.709842,-4.156317,-5.329916,-5.503359,2.102835,9


## Using Best Model From Grid Search

In [73]:
y_pred = grid_model.predict(scaled_X_test)

In [74]:
from sklearn.metrics import mean_squared_error

In [75]:
mean_squared_error(y_test, y_pred)

2.387342642087475